### Important information to run this code

* This code is written based on the data that is stored in this structure:

`<main_directory>/WAKEBENCH-<participant_id>/<benchmark_case_name>.<int>`

where `<int>` is an integer from 0 to 5 that tracks which 10-min period the data in that folder corresponds to. This file structure can be obtained with the following bash script:

```
#!/bin/bash
for fname in *; do
    if [ -d "$fname" ]; then
        casedir="$fname"
        echo $casedir
        idir=0
        for fname in $casedir/*uvw_3D*.nc; do
            echo $fname
            filelist=`echo $fname | sed s/_3D/_*D/`
            #echo $filelist
            newdir=${casedir}.${idir}
            mkdir $newdir
            cp -v $filelist $newdir/
            idir=$((idir+1))
        done
        mv $casedir original.$casedir
    fi
done
```

run from the directory `<main_directory>/WAKEBENCH-<participant_id>/` that contains one folder per benchmark case.

### load packages needed

In [1]:
import numpy as np
import pandas as pd 
import os, xarray
from glob import glob

import sys
sys.path.append("/home/pdoubraw/code/waketracking/")
from samwich.waketrackers import track
from samwich.dataloaders import XarrayData

Note: Install OpenCV library (cv2) to enable contour trackers, e.g., `conda install opencv`


### define some parameters

In [2]:
D = 27.
ref_area = np.pi/4 * D**2
z_hub = 32.1
npz_out_path = "/scratch/pdoubraw/"

### define some functions that will make things more automated

In [3]:
def get_cases(main_path='.'):
    """
    Parameters
    ----------
    main_path : string
        absolute path to participant directory
        
    Returns
    -------
    directory_list : list
        the benchmark case name directories available within the participant directory
    """
    return [ d for d in os.listdir(main_path) if os.path.isdir(os.path.join(main_path,d)) and not d.startswith('.')]

def get_distances(main_path,case):
    """    
    Get dictionary of downstream wake distances and a path to the sampled data
    
    The upstream datafile is returned separately, and assumed to only be at a single distance
    
    Parameters
    ----------
    main_path : string,
        absolute path to participant directory
    case : string,
        case name identified previously, which is a subdirectory within main_path
    """
    fpaths = sorted(glob(os.path.join(main_path,case,'*.nc')))
    dists  = []
    upD    = None
    uppath = None
    for fpath in fpaths:
        #print ("[...]/"+os.path.split(fpath)[-1])
        fname = os.path.split(fpath)[-1]
        assert fname.endswith('.nc')
        fname = fname[:-3] # strip .nc
        dist = fname.split('uvw_')[1].split('_')[0]
        #print(dist)
        assert dist.endswith('D')
        if float(dist[:-1]) < 0:
            assert upD is None
            upD = dist
            uppath = fpath
        else:
            dists.append(dist)
    fpaths.remove(uppath)
    return uppath, dict(zip(dists,fpaths))

def check_var_names(xa):
    """Since people cannot reliably save the netcdf data with consistent
    field names, we have to do an additional check (and if needed,
    rename) step.
    """
    nc_vars = xa.variables.keys()
    need_rename = False
    for var in ['t','x','y','z','u','v','w']:
        if var not in nc_vars:
            need_rename = True
            break
    if need_rename:
        name_dict = {}
        for variable_name in nc_vars:
            if variable_name.lower() == 'time':
                name_dict[variable_name] = 't'
            elif variable_name.lower() == variable_name:
                continue
            else:
                name_dict[variable_name] = variable_name.lower()
        print('renaming variables in netcdf file as follows:',name_dict)
        return xa.rename(name_dict=name_dict)

def estimate_inflow(data,outfile):
    """Given the streamwise velocity, estimate the inflow (i.e., the 
    freestream velocity to be removed from the wake velocity
    """
    # expected U.shape == (Ntimes, Ny, Nz)
    _,_,z,U = data.sliceI()
    assert np.all(z[0,:] == z[-1,:])
    zprofile = z[0,:]
    if len(U.shape) > 3:
        assert len(U.shape) == 4 # (Ntimes, Ny, Nz, 3)
        U = U[:,:,:,0]
    Uprofile = np.mean(U, axis=(0,1))
    np.savetxt(outfile, np.stack((zprofile,Uprofile),axis=-1))
    return zprofile,Uprofile

def track_with_Gaussian2D(data,profile,prefix):
    """Perform Gaussian2D wake tracking, given a precalculated inflow
    profile. Output is generated with the given prefix.
    """
    gauss2d = track(data.sliceI(), method='Gaussian2D', verbose=0)
    gauss2d.remove_shear(wind_profile=profile)
    ywake, zwake = gauss2d.find_centers(
        A_ref=ref_area,
        A_min=0.5*ref_area,
        A_max=2.0*ref_area,
        AR_max=3,
        trajectory_file=prefix+'trajectory.csv',
        outlines_file=prefix+'outlines.pkl',
    )
    return gauss2d

# start

In [4]:
# for interpolation into a meandering frame of reference, how big of a y-z grid do we want?
y_mfor = np.arange(-45,45.1,1)
z_mfor = np.arange(-40,40.1,1)

In [9]:
glob('/scratch/equon/wakebench/phase_2/WAKE*')

['/scratch/equon/wakebench/phase_2/WAKEBENCH-IFPEN_CATHELAIN_LES',
 '/scratch/equon/wakebench/phase_2/WAKEBENCH-NREL_DEBNATH_LES',
 '/scratch/equon/wakebench/phase_2/WAKEBENCH-NREL_SHALER_DWMT-LES',
 '/scratch/equon/wakebench/phase_2/WAKEBENCH-IFPEN_CATHELAIN_SSAM',
 '/scratch/equon/wakebench/phase_2/WAKEBENCH-NREL_SHALER_DWMT',
 '/scratch/equon/wakebench/phase_2/WAKEBENCH-FORWIND_KRUEGER_LES',
 '/scratch/equon/wakebench/phase_2/WAKEBENCH-SNL_MANIACI_LES',
 '/scratch/equon/wakebench/phase_2/WAKEBENCH-PPEGmbH_SCHLEZ_RANS',
 '/scratch/equon/wakebench/phase_2/WAKEBENCH-DTU_VANDERLAAN_RANS',
 '/scratch/equon/wakebench/phase_2/WAKEBENCH-DTU_VANDERLAAN_RANS_MOST']

In [5]:
main_paths = ['/scratch/equon/wakebench/phase_2/WAKEBENCH-NREL_SHALER_DWMT-LES']

# JUNK

In [16]:
for main_path in main_paths:
    
    cases     = get_cases(main_path=main_path)

    for case in cases:

        print('\n'+case+'\n-----------\n')

        ppid = os.path.split(main_path)[-1].split('WAKEBENCH-')[-1]

        upstream_fpath, wake_distances = get_distances(main_path, case)

        print('\n'+"upstream")
        
        # XarrayData is part of samwich (ewquon's code) dataloaders
        # use XarrayData to read upstream data and check that variable names are as expected
        upstream_data = XarrayData(upstream_fpath, check_vars=check_var_names)

        profile = estimate_inflow(upstream_data,
                          "./tmp.csv")

        # allocate space to keep wake objects at different downstream distances
        wake = {}
        for dist in sorted(wake_distances):
            ddown = float(dist.split('D')[0])
            fpath = wake_distances[dist]
            print('\n'+dist)
            # read wake data
            wakedata = XarrayData(fpath, check_vars=check_var_names)
            prefix = os.path.join(case, dist+'_')
            prefix = os.path.join(main_path,prefix)
            # initialize wake object and track it via the function defined above
            wake[ddown] = track_with_Gaussian2D(wakedata, profile, prefix)
            # also obtain data in meandering frame of reference
            wake[ddown].to_MFoR(y_mfor,z_mfor) 

        first = True
        for ddown in wake.keys():
            #
            # mfor
            #            
            
            # grab the nt,ny,nz array of u
            u          = np.squeeze(wake[ddown].u_mfor)
            # this u is NOT u_tot ... it is actually u_tot - u_freestream so backtrack the u_tot manually here
            # in the future u_tot_mfor will be implemented in samwich and this step won't be necessary
            if u.ndim == 3:
                nt, ny, nz = u.shape
            else:
                nt = 1
                ny, nz = u.shape

            
            u_tot = np.zeros((nt,ny,nz))                            
            for it in range(nt):
              
                wake_z = wake[ddown].xv_wake[it]
                tmp_z  = wake[ddown].xv_mfor[0,:] + wake_z
                tmp_u  = np.interp(tmp_z, profile[0], profile[1])
                
                if u.ndim ==3:
                    u_tot[it,...] = np.squeeze(u[it,...]) + tmp_u[np.newaxis,:]
                else:
                    u_tot[it,...] = u + tmp_u[np.newaxis,:]
                    
            u_tot = np.squeeze(u_tot)
    
            # get mean in time if there is a time dimension
            u     = np.mean(u_tot,axis=0) if u.ndim==3 else u_tot.copy()

            # get the horizontal profile of u_tot and y at the wake center
            k          = np.argmin(np.abs(wake[ddown].xv_mfor[0,:]))                        
            u1d_mfor_y = u[:,k]
            y1d        = wake[ddown].xh_mfor[:,k]            
            # get the vertical profile of u_tot and z at the wake center
            j          = np.argmin(np.abs(wake[ddown].xh_mfor[:,0]))                        
            u1d_mfor_z = u[j,:]
            z1d        = wake[ddown].xv_mfor[j,:]            
        
            
            #
            # ffor
            #
            u_ffor     = np.squeeze(wake[ddown].u_tot)
            u_ffor     = np.mean(u_ffor,axis=0) if u_ffor.ndim==3 else u_ffor.copy()

            # get the horizontal profile of u_tot and y at hub height
            k_ffor     = np.argmin(np.abs(wake[ddown].xv[0,:]-z_hub))
            u1d_ffor_y = u_ffor[:,k_ffor]
            y1d_ffor   = wake[ddown].xh[:,k_ffor]
            # get the vertical profile of u_tot and z at hub center in y
            j_ffor     = np.argmin(np.abs(wake[ddown].xh[:,0]))
            u1d_ffor_z = u_ffor[j_ffor,:]
            z1d_ffor   = wake[ddown].xv[j_ffor,:]
            
            if first:
                
                df_mfor_y = pd.DataFrame(index=y1d,columns=wake.keys())   
                df_mfor_z = pd.DataFrame(index=z1d,columns=wake.keys())                   

                df_ffor_y = pd.DataFrame(index=y1d_ffor,columns=wake.keys())   
                df_ffor_z = pd.DataFrame(index=z1d_ffor,columns=wake.keys())   

                first=False
            
            df_mfor_y[ddown] = u1d_mfor_y
            df_mfor_z[ddown] = u1d_mfor_z
            df_ffor_y[ddown] = u1d_ffor_y
            df_ffor_z[ddown] = u1d_ffor_z   
            
            # save 2d slices
            
            fout = os.path.join(npz_out_path,"{0}_mfor_mean_utot_{1}_{2}D.csv".format(ppid,case,ddown))
            print("===> {0}".format(fout))
            np.savez_compressed(fout,
                                u=u,
                                y=wake[ddown].xh_mfor,
                                z=wake[ddown].xv_mfor)

            fout = os.path.join(npz_out_path,"{0}_ffor_mean_utot_{1}_{2}D.csv".format(ppid,case,ddown))
            print("===> {0}".format(fout))
            np.savez_compressed(fout,
                                u=u_ffor,
                                y=wake[ddown].xh_mfor,
                                z=wake[ddown].xv_mfor)            
            
        # save the horizontal and vertical slices through wake center and hub center to csv files
        
        fout = os.path.join(npz_out_path,"{0}_{1}_mean_u_mfor_horline.csv".format(ppid,case))
        print("===> {0}".format(fout))
        df_mfor_y.to_csv(fout)            

        fout = os.path.join(npz_out_path,"{0}_{1}_mean_u_mfor_verline.csv".format(ppid,case))
        print("===> {0}".format(fout))
        df_mfor_z.to_csv(fout)            

        fout = os.path.join(npz_out_path,"{0}_{1}_mean_u_ffor_horline.csv".format(ppid,case))
        print("===> {0}".format(fout))
        df_ffor_y.to_csv(fout)            

        fout = os.path.join(npz_out_path,"{0}_{1}_mean_u_ffor_verline.csv".format(ppid,case))
        print("===> {0}".format(fout))
        df_ffor_z.to_csv(fout)                  
        
        # also save a npz file with the wake centers
                            
        for ddown in wake.keys():
            fout = os.path.join(npz_out_path,"{0}_wake_centers_{1}_{2}D.npz".format(ppid,case,ddown))
            print("===> {0}".format(fout))                    
            np.savez_compressed(fout,y=wake[ddown].xh_wake,z=wake[ddown].xv_wake)        
            


neutral
-----------


upstream
renaming variables in netcdf file as follows: {'time': 't', 'U': 'u', 'V': 'v', 'W': 'w'}
Slicing data at i=0 x=-66.0

1D
renaming variables in netcdf file as follows: {'time': 't', 'U': 'u', 'V': 'v', 'W': 'w'}
Slicing data at i=0 x=27.0
Note: wake tracking has already been performed
Interpolating with RectBivariateSpline


Transform: frame 3600



2D
renaming variables in netcdf file as follows: {'time': 't', 'U': 'u', 'V': 'v', 'W': 'w'}
Slicing data at i=0 x=54.0
Note: wake tracking has already been performed
Interpolating with RectBivariateSpline


Transform: frame 3600



3D
renaming variables in netcdf file as follows: {'time': 't', 'U': 'u', 'V': 'v', 'W': 'w'}
Slicing data at i=0 x=81.0
Note: wake tracking has already been performed
Interpolating with RectBivariateSpline


...

Transform: frame 2104

Transform: frame 2636

Transform: frame 3159

Transform: frame 3600


===> /scratch/pdoubraw/FORWIND_KRUEGER_LES_mfor_mean_utot_unstable_1.0D.csv
===> /scratch/pdoubraw/FORWIND_KRUEGER_LES_ffor_mean_utot_unstable_1.0D.csv
===> /scratch/pdoubraw/FORWIND_KRUEGER_LES_mfor_mean_utot_unstable_2.0D.csv
===> /scratch/pdoubraw/FORWIND_KRUEGER_LES_ffor_mean_utot_unstable_2.0D.csv
===> /scratch/pdoubraw/FORWIND_KRUEGER_LES_mfor_mean_utot_unstable_3.0D.csv
===> /scratch/pdoubraw/FORWIND_KRUEGER_LES_ffor_mean_utot_unstable_3.0D.csv
===> /scratch/pdoubraw/FORWIND_KRUEGER_LES_mfor_mean_utot_unstable_4.0D.csv
===> /scratch/pdoubraw/FORWIND_KRUEGER_LES_ffor_mean_utot_unstable_4.0D.csv
===> /scratch/pdoubraw/FORWIND_KRUEGER_LES_mfor_mean_utot_unstable_5.0D.csv
===> /scratch/pdoubraw/FORWIND_KRUEGER_LES_ffor_mean_utot_unstable_5.0D.csv
===> /scratch/pdoubraw/FORWIND_KRUEGER_LES_mfor_mean_utot_unstable_6.0D.csv
===> /scratch/pdoubraw/FORWIND_KRUEGER_LES_ffor_mean_utot_unstable_6.0D.csv
===> /scratch/pdoubraw/FORWIND_KRUEGER_LES_mfor_mean_utot_unstable_7.0D.csv
===> /scratc